# Using Astropy Units

In scientific computing, we often represent physical quantities as numbers:

In [1]:
distance_in_miles = 50
time_in_hours = 2
velocity_in_mph = distance_in_miles / time_in_hours
print(velocity_in_mph)

25.0


[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: ../particles/index.rst
[plasmapy.formulary]: ../formulary/index.rst

Representing a physical quantity as a number has risks. We might unknowingly perform operations with different units, like `time_in_seconds + time_in_hours`. We might even accidentally perform operations with physically incompatible units, like `length + time` without catching our mistake. We can avoid these problems by using a units package.

This notebook introduces [astropy.units] with an emphasis on the functionality needed to work with [plasmapy.particles] and [plasmapy.formulary]. We typically import this subpackage as `u`.

In [2]:
import astropy.units as u

## Contents

1. [Unit basics](#Unit-basics)
2. [Unit operations](#Unit-operations) 
3. [Unit conversations](#Unit-conversions)
4. [Detaching units and values](#Detaching-units-and-values)
5. [Equivalencies](#Equivalencies)
6. [Physical constants](#Physical-constants)
4. [Units in PlasmaPy](#Units-in-PlasmaPy)
5. [Optimizing unit operations](#Optimizing-unit-operations)
6. [Physical Types](#Physical-types)

## Unit basics



We can create a physical quantity by multiplying or dividing a number or array with a unit.

In [3]:
distance = 60 * u.km
print(distance)

60.0 km


[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

This operation creates a [Quantity]: a number, sequence, or array that has been assigned a physical unit.

In [4]:
type(distance)

astropy.units.quantity.Quantity

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can also create an object by using the [Quantity] class itself.

In [5]:
time = u.Quantity(120, u.min)

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can create [Quantity] objects with compound units.

In [6]:
88 * u.imperial.mile / u.hour

<Quantity 88. mi / h>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can even create [Quantity] objects that are dimensionless.

In [7]:
3 * u.dimensionless_unscaled

<Quantity 3.>

## Unit operations

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

Operations between [Quantity] objects handle unit conversions automatically. We can add [Quantity] objects together as long as their units have the same physical type.

In [8]:
1 * u.m + 25 * u.cm

<Quantity 1.25 m>

Units get handled automatically during operations like multiplication, division, and exponentiation.

In [9]:
velocity = distance / time
print(velocity)

0.5 km / min


In [10]:
area = distance ** 2
print(area)

3600.0 km2


Attempting an operation between physically incompatible units gives us an error, which we can use to find bugs in our code.

In [11]:
3 * u.m + 3 * u.s

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

## Unit conversions

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[.to]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to

The [.to] method allows us to convert a [Quantity] to different units of the same physical type. This method accepts strings that represent a unit (including compound units) or a unit object.

In [12]:
velocity.to("m/s")

<Quantity 8.33333333 m / s>

In [13]:
velocity.to(u.m / u.s)

<Quantity 8.33333333 m / s>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[.si]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.si
[.cgs]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.cgs

The [.si] and [.cgs] attributes convert the [Quantity] to SI or CGS units, respectively. 

In [14]:
velocity.si

<Quantity 8.33333333 m / s>

In [15]:
velocity.cgs

<Quantity 833.33333333 cm / s>

## Detaching units and values

[.value]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.value 
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

The [.value] attribute of a [Quantity] shows the number or array without the unit.

In [16]:
time.value

120.0

[.unit]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.unit
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

The [.unit] attribute of a [Quantity] shows the unit.

In [17]:
time.unit

Unit("min")

## Equivalencies

[electron-volt]: https://en.wikipedia.org/wiki/Electronvolt
[Boltzmann constant]: https://en.wikipedia.org/wiki/Boltzmann_constant

Plasma scientists often the [electron-volt] (eV) as a unit of temperature. This is a shortcut for describing the thermal energy per particle, or more accurately the temperature multiplied by the [Boltzmann constant], $k_B$. Because an electron-volt is a unit of energy rather than temperature, we cannnot directly convert electron-volts to kelvin.

In [18]:
u.eV.to("K")

UnitConversionError: 'eV' (energy/torque/work) and 'K' (temperature) are not convertible

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[equivalencies]: https://docs.astropy.org/en/stable/units/equivalencies.html
[temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

To handle situations like this, [astropy.units] has built-in [equivalencies]. The conversion from eV to K can be done by using the [temperature_energy()] equivalency.

In [19]:
(1 * u.eV).to("K", equivalencies=u.temperature_energy())

<Quantity 11604.51812155 K>

[dimensionless_angles()]: https://docs.astropy.org/en/stable/api/astropy.units.equivalencies.dimensionless_angles.html#dimensionless-angles

Radians are treated dimensionlessly by using the [dimensionless_angles()] equivalency. Note that this equivalency does not account for the multiplicative factor of $2π$ that is used when converting between frequency and angular frequency.

In [36]:
(3.2 * u.rad / u.s).to("1 / s", equivalencies=u.dimensionless_angles())

<Quantity 3.2 1 / s>

## Physical constants

[astropy.constants]: https://docs.astropy.org/en/stable/constants/index.html

We can use [astropy.constants] to access the most commonly needed physical constants.

In [21]:
from astropy.constants import c, e, k_B

print(c)

  Name   = Speed of light in vacuum
  Value  = 299792458.0
  Uncertainty  = 0.0
  Unit  = m / s
  Reference = CODATA 2018


[Constant]: https://docs.astropy.org/en/stable/api/astropy.constants.Constant.html#astropy.constants.Constant
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[u.temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

A [Constant] behaves very similarly to a [Quantity]. For example, we can use the Boltzmann constant to mimic the behavior of [u.temperature_energy()].

In [22]:
thermal_energy_per_particle = 0.6 * u.keV
temperature = thermal_energy_per_particle / k_B
print(temperature.to("MK"))

6.962710872930049 MK


Electromagnetic constants often need the unit system to be specified. Code within PlasmaPy generally uses SI units.

In [23]:
100 * e

TypeError: Constant 'e' does not have physically compatible units across all systems of units and cannot be combined with other values without specifying a system (eg. e.emu)

In [24]:
100 * e.si

<Quantity 1.60217663e-17 C>

## Units in PlasmaPy

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: ../particles/index.rst
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[Particle]: ../api/plasmapy.particles.particle_class.Particle.rst

Now we can show some uses of [astropy.units] in PlasmaPy.  We can start with [plasmapy.particles]. Many of the attributes of the [Particle] class provide [Quantity] objects.

In [25]:
from plasmapy.particles import Particle

alpha = Particle("He-4 2+")

In [26]:
alpha.charge

<Quantity 3.20435327e-19 C>

In [27]:
alpha.mass

<Quantity 6.64465719e-27 kg>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[plasmapy.formulary]: ../formulary/index.rst

Similarly, [Quantity] objects are the expected inputs and outputs of most functions in [plasmapy.formulary]. We can use them to calculate some plasma parameters for a typical region of the solar corona.

In [28]:
from plasmapy.formulary import Alfven_speed, gyrofrequency

In [29]:
B = 0.01 * u.T
n = 1e15 * u.m ** -3
proton = Particle("p+")

In [30]:
Alfven_speed(B=B, density=n, ion=proton).to("km /s")

<Quantity 6895.69343322 km / s>

In [31]:
gyrofrequency(B=B, particle="e-")

<Quantity 1.75882001e+09 rad / s>

## Optimizing unit operations

[performance tips]: https://docs.astropy.org/en/stable/units/index.html#performance-tips
[astropy.units]: https://docs.astropy.org/en/stable/units/index.html

Astropy's documentation includes [performance tips] for using [astropy.units] in computationally intensive applications. For example, in computationally intensive operations, it helps to put compound units in parentheses.

In [32]:
volume = 0.62 * (u.barn * u.Mpc)

## Physical types

[physical type]: https://docs.astropy.org/en/stable/units/physical_types.html
[physical_type]: https://docs.astropy.org/en/stable/api/astropy.units.UnitBase.html#astropy.units.UnitBase.physical_type
[get_physical_type()]: https://docs.astropy.org/en/stable/api/astropy.units.get_physical_type.html#astropy.units.get_physical_type

A [physical type] corresponds to physical quantities with dimensionally compatible units. Astropy has functionality that represents different physical types. These physical type objects can be accessed using either the [physical_type] attribute of a unit or [get_physical_type()].

In [33]:
(u.m ** 2 / u.s).physical_type

PhysicalType({'diffusivity', 'kinematic viscosity'})

In [34]:
u.get_physical_type("number density")

PhysicalType('number density')

These physical type objects can be used for dimensional analysis.

In [35]:
energy_density = (u.J * u.m ** -3).physical_type
velocity = u.get_physical_type("velocity")
print(energy_density * velocity)

energy flux/irradiance
